# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data/')
song_files = sorted(song_files)

In [6]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [7]:
df = pd.read_json(filepath, lines=True,encoding='utf-8')
df.head()

artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0

In [8]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [9]:
# Cleanup data by Column.  Year -> 1400, num_songs -> 0, duration, longitude and latitude -> 0.001
df['artist_id'].fillna("Missing artist id", inplace=True)
df['artist_latitude'].fillna(0.001, inplace=True)
df['artist_location'].fillna('Missing location', inplace=True)
df['artist_longitude'].fillna(0.001, inplace=True)
df['artist_name'].fillna("Missing artist name", inplace=True)
df['duration'].fillna(0.001, inplace=True)
df['num_songs'].fillna(0, inplace=True)
df['song_id'].fillna("Mising song id", inplace=True)
df['title'].fillna("Missing song title", inplace=True)
df['year'].fillna(1400, inplace=True)




In [10]:
df[:3]

artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1            0.001  California - LA             0.001   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = list(df.loc[:,['song_id','title','artist_id','year','duration']].values[0])
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = list(df.loc[:,['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', 0.001, 0.001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('data/log_data/')

In [16]:
log_files = sorted(log_files)

In [17]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-01-events.json


In [18]:
df = pd.read_json(filepath, lines=True)
df.head(2)

artist       auth firstName gender  itemInSession lastName  length level  \
0   None  Logged In    Walter      M              0     Frye     NaN  free   
1   None  Logged In    Kaylee      F              0  Summers     NaN  free   

                            location method  page   registration  sessionId  \
0  San Francisco-Oakland-Hayward, CA    GET  Home  1540919166796         38   
1        Phoenix-Mesa-Scottsdale, AZ    GET  Home  1540344794796        139   

   song  status             ts  \
0  None     200  1541105830796   
1  None     200  1541106106796   

                                           userAgent  userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      39  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

In [19]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
df = pd.read_json(filepath, lines=True)
df.head(2)

artist       auth firstName gender  itemInSession lastName  length level  \
0   None  Logged In    Walter      M              0     Frye     NaN  free   
1   None  Logged In    Kaylee      F              0  Summers     NaN  free   

                            location method  page   registration  sessionId  \
0  San Francisco-Oakland-Hayward, CA    GET  Home  1540919166796         38   
1        Phoenix-Mesa-Scottsdale, AZ    GET  Home  1540344794796        139   

   song  status             ts  \
0  None     200  1541105830796   
1  None     200  1541106106796   

                                           userAgent  userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      39  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

In [21]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head(2)

0   2018-11-01 20:57:10.796
1   2018-11-01 21:01:46.796
Name: ts, dtype: datetime64[ns]

In [22]:
time_data = (t.dt.time.values, t.dt.hour.values, t.dt.day.values, t.dt.week.values, t.dt.month.values,
             t.dt.year.values, t.dt.weekday.values)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))

In [23]:
time_df.head(2)

start_time  hour  day  week  month  year  weekday
0  20:57:10.796000    20    1    44     11  2018        3
1  21:01:46.796000    21    1    44     11  2018        3

In [24]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
user_df = pd.DataFrame(df.loc[:,['userId', 'firstName', 'lastName', 'gender', 'level']])
user_df.head(2)

userId firstName lastName gender level
0      39    Walter     Frye      M  free
1       8    Kaylee  Summers      F  free

In [27]:
user_df.count()

userId       15
firstName    15
lastName     15
gender       15
level        15
dtype: int64

In [28]:
# print(user_df[user_df['userId'] == '' ])
user_df.drop_duplicates(inplace=True)

In [29]:
user_df.dropna(subset=['userId'], inplace=True)

In [30]:
user_df.count()

userId       5
firstName    5
lastName     5
gender       5
level        5
dtype: int64

In [31]:
print(user_df.size)
user_df.columns

25


Index(['userId', 'firstName', 'lastName', 'gender', 'level'], dtype='object')

In [32]:
# import numpy as np
# user_df.replace(to_replace=[None], value=pd.np.nan, inplace=True)
# user_df['userId'].fillna(0, inplace=True)
# # user_df['userId'] = user_df['userId'].replace(r'\s+', 0, regex=True) 
# user_df['firstName'].fillna("Missing FirstName", inplace=True)
# user_df['lastName'].fillna("Missing LastName", inplace=True)
# user_df['gender'].fillna("Missing Gender", inplace=True)
# user_df['level'].fillna("Missing Level", inplace=True)

In [33]:
# user_df.fillna(value=pd.np.nan, inplace=True)
user_df.head(2)

userId firstName lastName gender level
0      39    Walter     Frye      M  free
1       8    Kaylee  Summers      F  free

In [34]:
for i, row in user_df.iterrows():
    try:
        cur.execute(user_table_insert, row)
        conn.commit()
    except Exception as e:
        print(e)

In [58]:
# user_df = user_df[user_df.firstName != "Missing firstName"]
# user_df = user_df[user_df.lastName != "Missing lastName"]
# user_df = user_df[user_df.gender != "Missing Gender"]

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [37]:
import time
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    try:
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
    except psycopg2.Error as e:
        results = False
#         print("ERROR: Select Song failed.")
#         print(e)
#     print(results)
        
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    mt = pd.to_datetime(row['ts'], unit='ms')
    
    # insert songplay record
    tm = pd.to_datetime(row['ts'], unit='ms')
    songplay_data = (tm.time().strftime("%H:%M:%S"),
                     row.userId, 
                     row.level, 
                     songid, 
                     artistid, 
                     row.sessionId, 
                     row.location, 
                     row.userAgent)
    try:
        cur.execute(songplay_table_insert, (songplay_data))
        conn.commit()
    except psycopg2.Error as e:
        print(e)
        pass
        

cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed
cursor already closed


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [36]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.